In [314]:
from sklearn.tree import DecisionTreeRegressor 
import pandas as pd
from sklearn.cross_validation import KFold
import sklearn as sk
import numpy as np

input_file = 'GDP by country and year.csv'
target_file = 'life expectancy by country and year.csv'
HIV_file = 'HIV rates by country and year.csv'
Income_file = 'Income per capita by country and year.csv'
Internet_file = 'Internet user rates by country and year.csv'
Continent_file = 'Continent.csv'
health_data = 'health_data.csv'

In [315]:
input_df = pd.read_csv(input_file)
target_df = pd.read_csv(target_file)
HIV_df = pd.read_csv(HIV_file)
Income_df = pd.read_csv(Income_file)
Internet_df = pd.read_csv(Internet_file)
Continent_df = pd.read_csv(Continent_file)
health_df = pd.read_csv(health_data)
health_df = health_df.iloc[:-5, :]

In [316]:
# input_df
health_df.head()

,country_name,metric,1960,1961,1962,1963,1964,1965,1966,1967,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Afghanistan,birth_rate,51.276000,51.374000,51.464000,51.544000,51.614000,51.668000,51.70300,51.71600,...,42.779000,41.560000,40.265000,38.940000,37.636000,36.396000,35.253000,34.225000,33.314000,NaN
1,Afghanistan,death_rate,32.403000,31.902000,31.415000,30.937000,30.464000,29.992000,29.52100,29.05000,...,10.075000,9.771000,9.475000,9.193000,8.927000,8.677000,8.445000,8.230000,8.034000,NaN
2,Afghanistan,pop_growth,1.813677,1.874003,1.932414,1.989785,2.046675,2.102528,2.12596,2.14225,...,2.718198,2.485315,2.525628,2.736886,2.983979,3.135546,3.164336,3.033473,2.802986,NaN
3,Afghanistan,pct_pop_rural,91.779000,91.492000,91.195000,90.890000,90.574000,90.250000,89.91500,89.57000,...,76.413000,76.054000,75.687000,75.311000,74.926000,74.532000,74.129000,73.718000,73.297000,72.868
4,Afghanistan,pct_pop_urban,8.221000,8.508000,8.805000,9.110000,9.426000,9.750000,10.08500,10.43000,...,23.587000,23.946000,24.313000,24.689000,25.074000,25.468000,25.871000,26.282000,26.703000,27.132


In [317]:
# null_data = target_df[target_df.isnull().any(axis=1)]

In [318]:
# null_data.head()

In [334]:
worldbank_life_exp = pd.read_csv('worldbank_life_exp.csv')
GDP_growth = pd.read_csv('GDP_Growth.csv')

In [336]:
GDP_growth.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,ABW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.654626,-6.881302,-5.653502,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,0.078039,-8.594256,-3.817986,-5.347977,-4.802675,-1.760010,-0.063514,NaN,NaN,NaN
2,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,13.740205,3.611368,21.020649,8.433290,6.113685,14.434741,1.959123,1.312531,1.519911,NaN
3,Angola,AGO,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,22.593054,13.817146,2.412870,3.407655,3.918597,5.155441,6.813586,4.804473,2.997704,NaN
4,Albania,ALB,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,5.900000,7.530000,3.350000,3.710000,2.550000,1.420000,1.110000,2.000000,2.560000,NaN


In [320]:
val_set = worldbank_life_exp[['Country Name', '1960', '2011', '2012', '2013', '2014']]

In [321]:
# val_set

In [322]:
val_set_final = val_set.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [323]:
countries = target_df['Country Name'].values

In [324]:
VALIDATION = val_set_final[val_set_final['Country Name'].isin(countries)]

In [325]:
VALIDATION.head()

,Country Name,1960,2011,2012,2013,2014
0,Aruba,65.569366,75.081390,75.205756,75.328659,75.451098
2,Afghanistan,32.328512,59.327951,59.679610,60.028268,60.374463
3,Angola,32.984829,51.059317,51.464000,51.866171,52.266878
4,Albania,62.254366,77.240585,77.443976,77.640463,77.830463
6,United Arab Emirates,52.243220,76.851195,77.024146,77.195634,77.368171


In [326]:
Target_Columns = ['Country Name']+ [str(year) for  year in range(1950,2017)]
target_df_final = pd.DataFrame()
for year in Target_Columns:
    if year in target_df.columns:
        target_df_final['%s' % (year,)] = target_df['%s' % (year,)]
    else:
        target_df_final['%s' % (year,)] = np.nan

In [ ]:
Target_Columns = ['Country Name']+ [str(year) for  year in range(1950,2017)]
target_df_final = pd.DataFrame()
for year in Target_Columns:
    if year in target_df.columns:
        target_df_final['%s' % (year,)] = target_df['%s' % (year,)]
    else:
        target_df_final['%s' % (year,)] = np.nan

In [327]:
copy = target_df_final.iloc[:, 1:].copy()

In [328]:
copy.iloc[21:24, :]

,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.452000,69.594220,69.703073,69.782049,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,78.934146,79.236585,79.388439,79.288537,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,64.694122,65.273805,65.848000,66.407756,NaN,NaN,NaN,NaN,NaN,NaN


In [329]:
copy.columns = range(1950, 2017)

In [330]:
copy.interpolate(axis=1, limit_direction='both', method = 'spline', order = 1, limit = 10000, inplace = True)

In [331]:
copy.iloc[21:24, :]

,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
21,59.361585,59.665109,59.968633,60.272156,60.575680,60.879204,61.182727,61.486251,61.789775,62.093299,...,69.452000,69.594220,69.703073,69.782049,69.471639,69.438967,69.406295,69.373623,69.340951,69.308279
22,65.181139,65.424323,65.667507,65.910691,66.153875,66.397058,66.640242,66.883426,67.126610,67.369794,...,78.934146,79.236585,79.388439,79.288537,80.015360,80.258544,80.501728,80.744912,80.988096,81.231280
23,35.660050,36.158187,36.656324,37.154462,37.652599,38.150737,38.648874,39.147011,39.645149,40.143286,...,64.694122,65.273805,65.848000,66.407756,66.046429,66.544567,67.042704,67.540841,68.038979,68.537116


In [332]:
target_df_final.iloc[:, 1:] = copy.values

In [333]:
target_df_final.head()

,Country Name,1950,1951,1952,1953,1954,1955,1956,1957,1958,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Aruba,66.079189,66.242535,66.405881,66.569227,66.732574,66.895920,67.059266,67.222612,67.385958,...,74.526756,74.675732,74.818146,74.953537,76.043308,76.206654,76.370001,76.533347,76.696693,76.860039
1,Afghanistan,25.343483,25.928843,26.514204,27.099564,27.684924,28.270285,28.855645,29.441005,30.026366,...,57.833829,58.225024,58.603683,58.970829,61.050465,61.635825,62.221185,62.806546,63.391906,63.977267
2,Angola,31.276143,31.543230,31.810316,32.077403,32.344489,32.611575,32.878662,33.145748,33.412835,...,49.435732,49.847390,50.251024,50.654171,49.593188,49.941265,50.289343,50.637420,50.985497,51.333575
3,Albania,62.088492,62.337358,62.586223,62.835089,63.083955,63.332821,63.581686,63.830552,64.079418,...,76.470293,76.652073,76.840366,77.036951,77.269299,77.518165,77.767030,78.015896,78.264762,78.513627
4,United Arab Emirates,50.418938,50.965479,51.512020,52.058562,52.605103,53.151644,53.698185,54.244726,54.791268,...,76.111171,76.307756,76.494780,76.675244,78.208385,78.532944,78.857502,79.182061,79.506619,79.831178


In [313]:
input_df.iloc[4:7, -20:]

,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
4,9.83732,3.184390,3.191836,-5.242922,1.635449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,12.66971,11.940775,5.906920,5.836201,-2.845210,5.52669,8.111047,3.850179,-3.385457,-0.788999,-4.40884,-10.894485,8.837041,9.029573,8.889308,8.142536,9.015745,4.093862,-6.013355,10.353567
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
input_df.fillna(0.0, inplace=True)

In [225]:
input_df.head()

,Country Name,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010
0,Aruba,1.245086,7.814432,6.666622,1.154469,4.514062,-0.364696,-2.164459,0.465290,7.519472,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,8.444163,1.055556,11.175270,5.554138,13.740205,3.611368,21.020649,8.433290,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Angola,2.800000,7.900000,5.600000,0.400000,-0.300000,-1.200000,-6.900000,-24.700000,3.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Albania,5.745635,2.948597,1.104938,-1.251597,1.780644,5.637243,-0.787843,-1.420040,9.836549,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,United Arab Emirates,16.526856,21.439330,-1.589603,20.923573,23.874775,4.659177,-6.719316,-4.745821,4.016951,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [226]:
np.sum(input_df.isnull())

Country Name    0
1961            0
1962            0
1963            0
1964            0
1965            0
1966            0
1967            0
1968            0
1969            0
1970            0
1971            0
1972            0
1973            0
1974            0
1975            0
1976            0
1977            0
1978            0
1979            0
1980            0
1981            0
1982            0
1983            0
1984            0
1985            0
1986            0
1987            0
1988            0
1989            0
1990            0
1991            0
1992            0
1993            0
1994            0
1995            0
1996            0
1997            0
1998            0
1999            0
2000            0
2001            0
2002            0
2003            0
2004            0
2005            0
2006            0
2007            0
2008            0
2009            0
2010            0
dtype: int64

In [227]:
target_data_long = pd.melt(target_df, id_vars=['Country Name'], var_name = 'Year', value_name='LifeExp')
input_data_long = pd.melt(input_df, id_vars=['Country Name'], var_name = 'Year', value_name='GDP')

In [228]:
target_data_long.head()

,Country Name,Year,LifeExp
0,Aruba,1961,65.988024
1,Afghanistan,1961,32.777439
2,Angola,1961,33.386220
3,Albania,1961,63.273463
4,United Arab Emirates,1961,53.286561


In [229]:
target_data_long.tail()

,Country Name,Year,LifeExp
10095,"Yemen, Rep.",2010,62.768732
10096,South Africa,2010,54.390756
10097,"Congo, Dem. Rep.",2010,56.896146
10098,Zambia,2010,56.383854
10099,Zimbabwe,2010,49.574659


In [230]:
input_data_long.head()

,Country Name,Year,GDP
0,Aruba,1961,1.245086
1,Afghanistan,1961,8.444163
2,Angola,1961,2.800000
3,Albania,1961,5.745635
4,United Arab Emirates,1961,16.526856


In [231]:
input_data_long.tail()

,Country Name,Year,GDP
10095,"Yemen, Rep.",2010,0.000000
10096,South Africa,2010,3.039747
10097,"Congo, Dem. Rep.",2010,7.078889
10098,Zambia,2010,10.298206
10099,Zimbabwe,2010,11.375921


In [232]:
np.sum(target_data_long.isnull())

Country Name    0
Year            0
LifeExp         0
dtype: int64

In [233]:
np.sum(input_data_long.isnull())

Country Name    0
Year            0
GDP             0
dtype: int64

In [235]:
X = input_data_long[['Country Name', 'Year', 'GDP']]
X.ix[:,'Year'] = pd.to_numeric(X.ix[:,'Year'])
Y = target_data_long[['LifeExp']]

In [236]:
X.head()

,Country Name,Year,GDP
0,Aruba,1961,1.245086
1,Afghanistan,1961,8.444163
2,Angola,1961,2.800000
3,Albania,1961,5.745635
4,United Arab Emirates,1961,16.526856


In [237]:
X.dtypes

Country Name     object
Year              int64
GDP             float64
dtype: object

In [238]:
Y.head()

,LifeExp
0,65.988024
1,32.777439
2,33.386220
3,63.273463
4,53.286561


In [240]:
VALIDATION_long = pd.melt(VALIDATION, id_vars=['Country Name'], var_name = 'Year', value_name='LifeExp')

In [242]:
VALIDATION.head()

,Country Name,1960,2011,2012,2013,2014
0,Aruba,65.569366,75.081390,75.205756,75.328659,75.451098
2,Afghanistan,32.328512,59.327951,59.679610,60.028268,60.374463
3,Angola,32.984829,51.059317,51.464000,51.866171,52.266878
4,Albania,62.254366,77.240585,77.443976,77.640463,77.830463
6,United Arab Emirates,52.243220,76.851195,77.024146,77.195634,77.368171


In [241]:
VALIDATION_long

,Country Name,Year,LifeExp
0,Aruba,1960,65.569366
1,Afghanistan,1960,32.328512
2,Angola,1960,32.984829
3,Albania,1960,62.254366
4,United Arab Emirates,1960,52.243220
5,Argentina,1960,65.215537
6,Armenia,1960,65.863463
7,Antigua and Barbuda,1960,61.782732
8,Australia,1960,70.817073
9,Austria,1960,68.585610


In [243]:
X

,Country Name,Year,GDP
0,Aruba,1961,1.245086
1,Afghanistan,1961,8.444163
2,Angola,1961,2.800000
3,Albania,1961,5.745635
4,United Arab Emirates,1961,16.526856
5,Argentina,1961,5.427843
6,Armenia,1961,-11.699998
7,Antigua and Barbuda,1961,4.184124
8,Australia,1961,2.475929
9,Austria,1961,5.537979


In [109]:
X = pd.get_dummies(X)

In [110]:
X.head()

,Year,GDP,Country Name_Afghanistan,Country Name_Albania,Country Name_Algeria,Country Name_Angola,Country Name_Antigua and Barbuda,Country Name_Argentina,Country Name_Armenia,Country Name_Aruba,...,Country Name_Uruguay,Country Name_Uzbekistan,Country Name_Vanuatu,"Country Name_Venezuela, RB",Country Name_Vietnam,Country Name_Virgin Islands (U.S.),Country Name_West Bank and Gaza,"Country Name_Yemen, Rep.",Country Name_Zambia,Country Name_Zimbabwe
0,1961,1.245086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1961,8.444163,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1961,2.800000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1961,5.745635,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1961,16.526856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Linear Regression

In [111]:
from sklearn import linear_model
linearRegression = linear_model.LinearRegression()

In [112]:
linearRegression.fit(X,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [113]:
predictions = linearRegression.predict(X)

In [114]:
print "MSE is" , sk.metrics.mean_squared_error(Y, predictions)

MSE is 9.33927835238


### Decision Trees

In [115]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
X, Y)
tree = DecisionTreeRegressor(max_depth = 230, random_state=0)
tree.fit(X_train, y_train)
predicted_y = tree.predict(X_train)
predicted_y_test = tree.predict(X_test)
print("MSE on training set: {:.3f}".format(sk.metrics.mean_squared_error(y_train, predicted_y)))
print("MSE on test set: {:.3f}".format(sk.metrics.mean_squared_error(y_test, predicted_y_test)))
print("R^2 on training set: {:.3f}".format(tree.score(X_train, y_train))) 
print("R^2 on test set: {:.3f}".format(tree.score(X_test, y_test)))

MSE on training set: 0.000
MSE on test set: 9.052
R^2 on training set: 1.000
R^2 on test set: 0.930


In [116]:
depth_list = range(200,240)
MSE_list = []

In [117]:
kf = KFold(len(Y), n_folds=10, shuffle = True)

In [118]:
for depth in depth_list:
    
    MSE_depth_CV =[]

    for train_index, test_index in kf:
        X_train_CV, X_test_CV = X.iloc[train_index,:], X.iloc[test_index,:]
        Y_train_CV, Y_test_CV = Y.iloc[train_index,:], Y.iloc[test_index,:]
        regressor = DecisionTreeRegressor(max_depth = depth) 
        regressor.fit(X_train_CV, Y_train_CV) 
        predicted_y = regressor.predict(X_test_CV)
        MSE = sk.metrics.mean_squared_error(Y_test_CV, predicted_y)
        MSE_depth_CV.append(MSE)
    
    print depth, "k-fold CV is", np.mean(MSE_depth_CV)

200 k-fold CV is 5.02219418121
201 k-fold CV is 4.9896153985
202 k-fold CV is 4.97982459383
203 k-fold CV is 4.9425572696
204 k-fold CV is 4.98157109792
205 k-fold CV is 4.90759559797
206 k-fold CV is 4.96292484338
207 k-fold CV is 4.97110931343
208 k-fold CV is 4.99948358212
209 k-fold CV is 4.95426815716
210 k-fold CV is 4.97129427406
211 k-fold CV is 4.92941253929
212 k-fold CV is 4.95527040572
213 k-fold CV is 4.94857114602
214 k-fold CV is 5.01951552862
215 k-fold CV is 4.93895000939
216 k-fold CV is 4.96845289681
217 k-fold CV is 4.99948278357
218 k-fold CV is 5.01802138704
219 k-fold CV is 4.95279391801
220 k-fold CV is 4.96584508872
221 k-fold CV is 4.95176764498
222 k-fold CV is 4.97411780814
223 k-fold CV is 4.98844886538
224 k-fold CV is 5.01498569965
225 k-fold CV is 5.00535289673
226 k-fold CV is 4.97889788729
227 k-fold CV is 4.9893457191
228 k-fold CV is 5.01774946152
229 k-fold CV is 4.96748485494
230 k-fold CV is 4.96624278261
231 k-fold CV is 4.96720360985
232 k-fold 

In [120]:
regressor = DecisionTreeRegressor(max_depth = 230) 
regressor.fit(X, Y) 
predicted_y = regressor.predict(X)
MSE = sk.metrics.mean_squared_error(Y, predicted_y)

print "MSE is", MSE

MSE is 6.95831034634e-11


### Random Forests

In [129]:
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(X, Y)
forest = RandomForestRegressor()

forest.fit(X_train, y_train.values.ravel())

predicted_y = forest.predict(X_train)
predicted_y_test = forest.predict(X_test)
print("MSE on training set: {:.3f}".format(sk.metrics.mean_squared_error(y_train, predicted_y)))
print("MSE on test set: {:.3f}".format(sk.metrics.mean_squared_error(y_test, predicted_y_test)))

print("R^2 on training set: {:.3f}".format(forest.score(X_train, y_train))) 
print("R^2 on test set: {:.3f}".format(forest.score(X_test, y_test)))

MSE on training set: 1.292
MSE on test set: 6.148
R^2 on training set: 0.990
R^2 on test set: 0.953


In [131]:
regressor = RandomForestRegressor() 
regressor.fit(X, Y.values.ravel()) 
predicted_y = regressor.predict(X)
MSE = sk.metrics.mean_squared_error(Y, predicted_y)

print "MSE is", MSE

MSE is 0.750381423294


### Gradient Boosting Machines

In [134]:
from sklearn.ensemble import GradientBoostingRegressor
X_train, X_test, y_train, y_test = train_test_split(
        X,Y)
gbrt = GradientBoostingRegressor(max_depth=40)
gbrt.fit(X_train, y_train.values.ravel())

predicted_y = gbrt.predict(X_train)
predicted_y_test = gbrt.predict(X_test)
print("MSE on training set: {:.3f}".format(sk.metrics.mean_squared_error(y_train, predicted_y)))
print("MSE on test set: {:.3f}".format(sk.metrics.mean_squared_error(y_test, predicted_y_test)))

MSE on training set: 0.445
MSE on test set: 2.341


In [80]:
?GradientBoostingClassifier

Object `GradientBoostingClassifier` not found.


In [133]:
regressor = GradientBoostingRegressor(max_depth=40)
regressor.fit(X, Y.values.ravel()) 
predicted_y = regressor.predict(X)
MSE = sk.metrics.mean_squared_error(Y, predicted_y)

print "MSE is", MSE

MSE is 0.501919205554


### Neural Networks

In [82]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.neural_network import MLPRegressor
# scaler = StandardScaler()

# X_scaled = scaler.fit_transform(X)

ImportError: cannot import name MLPRegressor

In [ ]:
# mlp = MLPReg(max_iter=1000, random_state=0)
# mlp.fit(X_train_scaled, y_train)
# print("Accuracy on training set: {:.3f}".format( mlp.score(X_train_scaled, y_train)))
# print("Accuracy on test set: {:.3f}".format(mlp.score(X_test_scaled, y_test)))

In [ ]:
# X = input_data_long[['Year', 'GDP']]
# Y = target_data_long[['LifeExp']]

In [323]:
# X = pd.get_dummies(X)
# X.ix[:,'Year'] = pd.to_numeric(X.ix[:,'Year']).values

In [325]:
# X.head()

,Year,GDP
0,1961,1.245086
1,1961,8.444163
2,1961,2.800000
3,1961,5.745635
4,1961,16.526856


In [326]:
# X.dtypes

Year      int64
GDP     float64
dtype: object

In [327]:
# linearRegression.fit(X,Y)
# predictions = linearRegression.predict(X)
# print "MSE is" , sk.metrics.mean_squared_error(Y, predictions)

MSE is 112.200475408


In [328]:
# print linearRegression.coef_

[[ 0.28348918 -0.05335729]]


In [491]:
# print("scikit-learn version: {}".format(sk.__version__))

scikit-learn version: 0.17.1


### AdaBoost Regressor

,LifeExp
0,65.988024
1,32.777439
2,33.386220
3,63.273463
4,53.286561


In [141]:
from sklearn.ensemble import AdaBoostRegressor
rate = [0.1]    # Learning rate; default is 1.0

for r in rate:
        regressor = AdaBoostRegressor() 
        regressor.fit(X, Y.values.ravel())
        predictions = regressor.predict(X)
        print "MSE with learning rate, %s is" % (r) , sk.metrics.mean_squared_error(Y, predictions) 
        print predictions
        print Y

MSE with learning rate, 0.1 is 116.370505845
[ 55.92313653  55.7170374   55.83168702 ...,  66.14306474  66.14306474
  63.35901665]
         LifeExp
0      65.988024
1      32.777439
2      33.386220
3      63.273463
4      53.286561
5      65.338512
6      66.284390
7      62.195415
8      70.973171
9      69.577317
10     61.239171
11     41.545463
12     70.520976
13     37.731195
14     34.938634
15     46.457927
16     70.195610
17     53.458512
18     63.072512
19     60.941585
20     68.212659
21     60.512780
22     68.897805
23     42.433439
24     54.718707
25     61.215098
26     63.048951
27     32.731024
28     50.907634
29     36.895707
...          ...
10070  73.197561
10071  72.307439
10072  49.862073
10073  57.281561
10074  73.694366
10075  68.563390
10076  65.021244
10077  67.308390
10078  72.182610
10079  69.797561
10080  74.602439
10081  74.090927
10082  61.625610
10083  55.836659
10084  70.265366
10085  76.394610
10086  78.541463
10087  67.861293
10088  72.346390
10

In [152]:
regressor = RandomForestRegressor() 
regressor.fit(X, Y.values.ravel()) 
predictions = regressor.predict(X)
MSE = sk.metrics.mean_squared_error(Y, predictions)

print "MSE is", MSE

MSE is 0.756749663491


In [37]:
X.head()

,Year,GDP,Country Name_Afghanistan,Country Name_Albania,Country Name_Algeria,Country Name_Angola,Country Name_Antigua and Barbuda,Country Name_Argentina,Country Name_Armenia,Country Name_Aruba,...,Country Name_Uruguay,Country Name_Uzbekistan,Country Name_Vanuatu,"Country Name_Venezuela, RB",Country Name_Vietnam,Country Name_Virgin Islands (U.S.),Country Name_West Bank and Gaza,"Country Name_Yemen, Rep.",Country Name_Zambia,Country Name_Zimbabwe
0,1961,1.245086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1961,8.444163,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1961,2.800000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1961,5.745635,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1961,16.526856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X_train = df.iloc[:,0:280]
X_test = test_df.iloc[:,0:280]

In [5]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40.30467,53.845657,0.0,401.0,15.0,15.52416,32.44188,0.0,377.0,3.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
Y_train = df.iloc[:,280:281]
Y_test = test_df.iloc[:,280]

In [7]:
depth = 5
regressor = DecisionTreeRegressor(max_depth=depth) 
regressor.fit(X_train, Y_train) 
predicted_y = regressor.predict(X_test)

In [8]:
sk.metrics.mean_squared_error(Y_test, predicted_y)

669.66569160245422

In [9]:
depth_list = range(1,20)
MSE_list = []

In [32]:
kf = KFold(len(Y_train), n_folds=10, shuffle = True)

In [27]:
print len(X_train), len(Y_train)

52397 52397


In [33]:
for depth in depth_list:
    
    MSE_depth_CV =[]

    for train_index, test_index in kf:
        X_train_CV, X_test_CV = X_train.iloc[train_index,:], X_train.iloc[test_index,:]
        Y_train_CV, Y_test_CV = Y_train.iloc[train_index,:], Y_train.iloc[test_index,:]
        regressor = DecisionTreeRegressor(max_depth=depth) 
        regressor.fit(X_train_CV, Y_train_CV) 
        predicted_y = regressor.predict(X_test_CV)
        MSE = sk.metrics.mean_squared_error(Y_test_CV, predicted_y)
        MSE_depth_CV.append(MSE)
    
    print depth, "k-fold CV is", np.mean(MSE_depth_CV)

1 k-fold CV is 1094.48596811
2 k-fold CV is 816.360036677
3 k-fold CV is 723.863492028
4 k-fold CV is 694.557312437
5 k-fold CV is 722.944608945
6 k-fold CV is 751.824008538
7 k-fold CV is 770.717029321
8 k-fold CV is 846.656690323
9 k-fold CV is 905.152489712
10 k-fold CV is 966.206092789
11 k-fold CV is 1030.32243944
12 k-fold CV is 1053.06401763
13 k-fold CV is 1099.489705
14 k-fold CV is 1081.98415069
15 k-fold CV is 1121.63768719
16 k-fold CV is 1090.79088817
17 k-fold CV is 1090.04682546
18 k-fold CV is 1082.8004027
19 k-fold CV is 1119.92789668


In [16]:
from StringIO import StringIO
import pydotplus 

my_tree = DecisionTreeRegressor(max_depth=5).fit(X_train_CV, Y_train_CV)
dotfile = StringIO()
sk.tree.export_graphviz(my_tree, out_file=dotfile)
graph = pydotplus.graph_from_dot_data(dotfile.getvalue())
graph.write_pdf("tree.pdf")

True

In [20]:
from IPython.display import IFrame
IFrame("tree.pdf", width=600, height=300)

# Compare to Linear Regression

In [31]:
from sklearn import linear_model

my_tree = DecisionTreeRegressor(max_depth=5).fit(X_train, Y_train)

algo = linear_model.LinearRegression()
algo.fit (X_train, Y_train)
predictions_LR = algo.predict(X_test)
predictions_DT = my_tree.predict(X_test)

MSE_LR = sk.metrics.mean_squared_error(Y_test, predictions_LR)
MSE_DT = sk.metrics.mean_squared_error(Y_test, predictions_DT)

print "MSE on the testing set using Linear Regression is ", MSE_LR
print "MSE on the testing set using the Decision Tree is ", MSE_DT

MSE on the testing set using Linear Regression is  1632.13874612
MSE on the testing set using the Decision Tree is  669.665691602


In [42]:
ridge = linear_model.Ridge (alpha = 0.1)
lasso = linear_model.Lasso(alpha = 0.1)

for model in [ridge, lasso]:
    model.fit(X_train, Y_train)
    predictions = model.predict(X_test)
    
    print "MSE on the testing set using %s is " % (model) , sk.metrics.mean_squared_error(Y_test, predictions)

MSE on the testing set using Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001) is  1631.67077
MSE on the testing set using Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False) is  1644.88424921
